In [ ]:
# Importação das Bibliotecas Necessárias
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import h5py

In [ ]:
# Carregamento dos Dados
with h5py.File('/content/drive/MyDrive/data/combined_data.h5', 'r') as h5f:
    X_data = h5f['X'][:]
    Y_data = h5f['Y'][:]

In [ ]:
# Pré-processamento dos Dados
X_data = X_data.astype('float32') / 255.0
Y_data = Y_data.astype('float32') / 255.0
X_train, X_test = X_data[:int(len(X_data)*0.8)], X_data[int(len(X_data)*0.8):]
Y_train, Y_test = Y_data[:int(len(Y_data)*0.8)], Y_data[int(len(Y_data)*0.8):]

In [ ]:
# U-Net
def unet_model(input_size=(128, 128, 3)):
    inputs = Input(input_size)

    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)

    # Decoder
    up6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(conv5)
    merge6 = concatenate([conv4, up6], axis=3)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)
    conv9 = Conv2D(2, 3, activation='relu', padding='same')(conv9)
    conv10 = Conv2D(1, 1, activation='sigmoid')(conv9)

    model = Model(inputs, conv10)

    return model

In [ ]:
# Compilação e Treinamento do Modelo
model = unet_model()
model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, Y_train, epochs=50, batch_size=16, validation_data=(X_test, Y_test), verbose=1)

# Avaliação do Modelo
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Perda: {loss}")
print(f"Acurácia: {accuracy}")

# Visualização dos Resultados da Segmentação
predictions = model.predict(X_test[:5])

for i in range(5):
    plt.figure(figsize=(10, 4))

    # Imagem original
    plt.subplot(1, 3, 1)
    plt.imshow(X_test[i])
    plt.title("Imagem Original")

    # Máscara real
    plt.subplot(1, 3, 2)
    plt.imshow(Y_test[i].reshape(128, 128), cmap='gray')
    plt.title("Máscara Real")

    # Máscara predita
    plt.subplot(1, 3, 3)
    plt.imshow(predictions[i].reshape(128, 128), cmap='gray')
    plt.title("Máscara Predita")

    plt.show()